In [ ]:
import gym
import random
import numpy as np
import tensorflow as tf
from collections import deque

In [ ]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model =tf.keras.Sequential([
            tf.keras.layers.Dense(24, input_dim=self.state_size, activation='relu'),
            tf.keras.layers.Dense(24, activation='relu'),
            tf.keras.layers.Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=self.learning_rate))
        return model
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
batch_size = 32
if __name__ == "__main__":
    # initialize gym environment and agent
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)

    # run for 1000 episodes
    for e in range(1000):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        done = False
        time = 0
        while not done:
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            time += 1
            if done:
                print("episode: {}/{}, score: {}, e: {:.2}"
                      .format(e, 1000, time, agent.epsilon))
                break
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 30ms/step
episode: 308/1000, score: 169, e: 0.21
1/1 [==============================] - 0s 21ms/step
episode: 309/1000, score: 145, e: 0.21
1/1 [==============================] - 0s 26ms/step
episode: 310/1000, score: 52, e: 0.21
1/1 [==============================] - 0s 20ms/step
episode: 311/1000, score: 78, e: 0.21
1/1 [==============================] - 0s 28ms/step
episode: 312/1000, score: 367, e: 0.21
1/1 [==============================] - 0s 20ms/step
episode: 313/1000, score: 174, e: 0.21
1/1 [==============================] - 0s 19ms/step
episode: 314/1000, score: 182, e: 0.21
1/1 [==============================] - 0s 22ms/step
episode: 315/1000, score: 74, e: 0.21
1/1 [==============================] - 0s 22ms/step
episode: 316/1000, score: 204, e: 0.21
1/1 [==============================] - 0s 20ms/step
episode: 317/1000, score: 252, e: 0.21
1/1 [==============================] - 0s 